In [1]:
import os

In [3]:
%pwd

'c:\\Users\\poppo\\Desktop\\github ai\\end to end by vivek\\TEXT-SUMMARIZER-PROJECT-ETE\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\poppo\\Desktop\\github ai\\end to end by vivek\\TEXT-SUMMARIZER-PROJECT-ETE'

In [6]:


from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path



In [7]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories




In [8]:


class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config



In [9]:


import os
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk



c:\Users\poppo\anaconda3\envs\mlproj\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-01-13 19:22:32,525: INFO: config: PyTorch version 2.1.2 available.]
[2024-01-13 19:22:32,530: INFO: config: TensorFlow version 2.12.0 available.]


In [10]:


class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)


    
    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )
            
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))



In [11]:


try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e



[2024-01-13 19:22:53,236: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-13 19:22:53,240: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-13 19:22:53,245: INFO: common: created directory at: artifacts]


BoxKeyError: "'ConfigBox' object has no attribute 'data_transformation'"